In [ ]:
%%capture install
%pip install manim
%pip install IPython --upgrade

In [2]:
from manim import *
import numpy as np
import scipy.integrate as integrate

Manim Community v0.17.2

In [3]:
%%manim -qh -v WARNING Day2

class Day2(MovingCameraScene):
    def construct(self):
        ax = Axes(
            x_range=[-5, 5, 1], y_range=[-2, 16, 2], axis_config={"include_tip": False}, x_axis_config={"numbers_to_include": np.arange(-5, 5.1, 2)},
            y_axis_config={"numbers_to_include": np.arange(0, 16, 2)}
        )
        parabola = ax.plot(lambda x: x**2).set_color(RED)

        labels = ax.get_axis_labels(x_label="x", y_label=MathTex(r"f(x)"))

        graph_label = ax.get_graph_label(
            parabola, MathTex(r"x^2"), x_val=4, direction=LEFT / 2
        )

        chart = VGroup(ax, parabola, labels, graph_label)
        self.play(Create(chart))
        
        bar_count = 5
        bar_count_var = Variable(bar_count, Text("# of bars", font_size=24), var_type=Integer).shift(RIGHT * 8 + UP*2.5)
        bar_count_tracker = bar_count_var.tracker
        self.play(Write(bar_count_var))
        self.wait(3)

        def maths():
            heights = [round(x**2, 3) for x in np.arange(0, 5, 5/bar_count)]
            dx = round(5/bar_count, 2)
            total = sum(dx*i for i in heights)
            maths = [rf"{dx}" + r"\cdot" + rf"{h}" for h in heights]
            maths.append(r'\text{Total } = ' + rf'{total}')
            math_tex = MathTex(r'\\'.join(maths)).scale(0.6).shift(RIGHT*9.5 + DOWN*1.2)
            self.play(Write(math_tex))
            return math_tex

        def make_bars():
            bars = ax.get_riemann_rectangles(
                graph=parabola,
                x_range=[0, 5],
                dx=5/bar_count,
            )
            self.play(Create(bars), run_time=5)
            return bars

        self.play(self.camera.frame.animate.shift(RIGHT*5))

        TOP_RIGHT = [9.5, 3.5, 0]

        x = Text(
            "Area under curve can be approximated\nby area of rectangles.",
            font_size=16
        ).move_to(TOP_RIGHT)
        self.play(Write(x))

        bars = make_bars()

        # draw lines from top of rectangle
        GRAPH_ORIGIN = np.array([0, -2, 0])

        answer = round(integrate.quad(lambda x: x**2, 0, 5)[0], 2)
        self.play(Write(Text("Actual:", font_size=24).shift(RIGHT*7.5 + UP*1.5)))
        self.play(Write(Text("Estimated:", font_size=24).shift(RIGHT*9.5 + UP*1.5)))
        actual = MathTex(r'\int_{0}^{5} x^2 \, dx =' + rf'{answer}').scale(0.6).shift(RIGHT * 7.5)
        self.play(Write(actual))

        math_tex = maths()

        self.wait(5)

        self.play(Uncreate(bars), Unwrite(math_tex), run_time=3)

        bar_count = 10
        self.play(bar_count_tracker.animate.set_value(bar_count))
        bars = make_bars()

        math_tex = maths()

        self.wait(5)
